In [1]:
import pandas as pd
import numpy as np

# data = np.load('distorted_data/concatenated_data.npz')
data = np.load('datasets/training.npz')
print(len(data['original']), list(data.keys()))
data

# # load the data from every file in the distorted_data directory

# import os
# data_dir = 'distorted_data'

# data = []
# for filename in os.listdir(data_dir):
#     if filename.endswith('.npz'):
#         file_path = os.path.join(data_dir, filename)
#         loaded_data = np.load(file_path)
#         data.append(loaded_data)

# # concatenate the data from all files
# all_data = {}
# for key in data[0].keys():
#     all_data[key] = np.concatenate([d[key] for d in data], axis=0)

# # check the shape of the concatenated data
# for key, value in all_data.items():
#     print(f"{key}: {value.shape}")

# # save the concatenated data to a new npz file
# output_file = 'distorted_data/concatenated_data.npz'
# np.savez(output_file, **all_data)

34561 ['original', 'label', 'Uniform_Noise', 'Rotate_90deg']


In [2]:
import os

def import_data(directory, save_path=None, save=False):

    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.npz'):
            file_path = os.path.join(directory, filename)
            loaded_data = np.load(file_path)
            data.append(loaded_data)

    # concatenate the data from all files
    all_data = {}
    for key in data[0].keys():
        all_data[key] = np.concatenate([d[key] for d in data], axis=0)

    # check the shape of the concatenated data
    for key, value in all_data.items():
        print(f"{key}: {value.shape}")  

    if save:
        if save_path is None:
            save_path = f'datasets/{directory}_concatenated_data.npz'
        np.savez(save_path, **all_data)
        print(f"Data saved to {save_path}")

    return all_data

In [3]:
train_data = import_data('imported_distorted/train', save_path='datasets/train.npz', save=True)
val_data =  import_data('imported_distorted/val', save_path='datasets/validation.npz', save=True)
test_data = import_data('imported_distorted/test', save_path='datasets/test.npz', save=True)

original: (34561, 224, 224)
label: (34561, 1)
Uniform_Noise: (34561, 224, 224)
Rotate_90deg: (34561, 224, 224)
Data saved to datasets/train.npz
original: (6491, 224, 224)
label: (6491, 1)
Uniform_Noise: (6491, 224, 224)
Rotate_90deg: (6491, 224, 224)
Data saved to datasets/validation.npz
original: (17778, 224, 224)
label: (17778, 1)
Uniform_Noise: (17778, 224, 224)
Rotate_90deg: (17778, 224, 224)
Data saved to datasets/test.npz


In [4]:
test_data.keys()

dict_keys(['original', 'label', 'Uniform_Noise', 'Rotate_90deg'])

In [5]:
# keys = list(data.keys())

# images = [data[keys[0]], data[keys[2]], data[keys[3]]]
# labels = data[keys[1]]

In [6]:
# images[0][0]

In [7]:
def normalize_image(image, mean=0.5, std=0.5):
    """
    Normalize an image tensor to have a mean and standard deviation.
    """
    return (image - mean) / std

def normalize_images(images, mean=0.5, std=0.5):
    """
    Normalize a list of images.
    """
    return [normalize_image(image, mean, std) for image in images]

In [8]:
# normalized_images = []
# for image in images:
#     normalized_images.append(normalize_images(image))

In [9]:
def plot_sample_images(images, labels, n=3):
    """
    Plot a sample of images with their corresponding labels.
    """
    import matplotlib.pyplot as plt
    import numpy as np

    plt.figure(figsize=(15, 5))
    for i in range(n):
        plt.subplot(1, n, i + 1)
        plt.imshow(images[i][0], cmap='gray')
        plt.title(f'Label: {labels[i]}')
        plt.axis('off')
    plt.show()

# plot_sample_images(images, labels)
# plot_sample_images(normalized_images, labels)

In [10]:
# images[0][0]

In [11]:
# normalized_images[0][0]

In [12]:
# type(normalized_images[0][0])
# normalized_images[0][0].shape

In [13]:
# zero_labels = np.zeros_like(labels)
# one_labels = np.ones_like(labels)

# print(zero_labels.shape, one_labels.shape)

# domain_labels = np.concatenate((zero_labels, one_labels, one_labels), axis=0)
# print(domain_labels.shape)

# expanded_labels = np.concatenate((labels, labels, labels), axis=0)
# print(expanded_labels.shape)

In [14]:
# # concatenate the matrices in the images list
# concatenated_images = np.concatenate(normalized_images, axis=0)
# concatenated_images.shape

In [15]:
from branched_resnet import CustomImageDataset

def preprocess_data(data, distortions = [3]):

    keys = list(data.keys())

    images = [data[keys[0]]]

    for distortion in distortions:
        images.append(data[keys[distortion]])

    labels = data[keys[1]]

    normalized_images = []
    for image in images:
        normalized_images.append(normalize_images(image))

    zero_labels = np.zeros_like(labels)
    one_labels = np.ones_like(labels)

    domain_label_list = [zero_labels]
    expanded_label_list = [labels]

    for _ in distortions:
        domain_label_list.append(one_labels)
        expanded_label_list.append(labels)

    domain_labels = np.concatenate(domain_label_list, axis=0)
    expanded_labels = np.concatenate(expanded_label_list, axis=0)

    concatenated_images = np.concatenate(normalized_images, axis=0)

    dataset = CustomImageDataset(images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

    return dataset

In [ ]:
import branched_resnet as br 
from branched_resnet import CustomImageDataset
from transformers import Trainer, TrainingArguments, PreTrainedModel, ResNetConfig

config = ResNetConfig()
model = br.ResNetForMultiLabel(config=config, num_d1_classes=11, num_d2_classes=2)

for name, param in model.named_parameters():
    print(name, param.data.mean(), param.requires_grad)
    break

#train_ds = CustomImageDataset(images=concatenated_images, labels1=expanded_labels, labels2=domain_labels)

# vars(model)

resnet.embedder.embedder.convolution.weight tensor(6.4683e-06) True


In [ ]:
train_ds = preprocess_data(train_data, distortions=[3])
val_ds = preprocess_data(val_data, distortions=[])
#test_ds = preprocess_data(test_data, distortions=[2, 3])

# D20

Train on undistorted and rotate 90

Test on the rest

In [ ]:
trainer = br.train_model(train_dataset=train_ds, eval_dataset= val_ds, model=model, output_dir= "./D20_Experiment", num_epochs=50, batch_size=200)
trainer.save_model('branched_resnet_model_D20')

C:\Users\samue\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: samuelsavine (samuelsavine-johns-hopkins-university). Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Accuracy Branch1,Precision Branch1,Recall Branch1,F1 Branch1,Accuracy Branch2,Precision Branch2,Recall Branch2,F1 Branch2
1,0.164000,0.265856,0.907872,0.898465,0.891680,0.891518,0.004468,0.500000,0.002234,0.004448


KeyboardInterrupt: 